<a href="https://colab.research.google.com/github/PKuznetsov-vl/corporate/blob/dev/Corporate_Network_algorithm_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipython-autotime
%reload_ext autotime
import autotime
import math
import time
import pandas as pd
import numpy as np
import gc
from numpy import linalg as LA
from google.colab import drive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 1.05 s (started: 2022-08-31 23:02:13 +00:00)


In [ ]:
SH_MODE = 1 # only SH nodes are considered final holders
LEVELS = 20 # this parameter should be equal or exceed the number of "onion layers" in data. 20 is a bit of overkill for safety

autotime.TIME_FORMAT = '%d/%m/%Y %H:%M:%S'
autotime.RUNNING_FORMAT = '{timespan} ({start})'
autotime.FINISHED_FORMAT = '{timespan} ({start} ~ {end})'

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 1.73 s (started: 2022-08-31 23:02:17 +00:00)


In [ ]:
path = '/content/drive/MyDrive/egrul_share.csv'
 
data=pd.read_csv(path,usecols={'founder_inn', 'inn', 'capital_p'},
               dtype={'founder_inn': str,'inn': str, 'capital_p':float})

time: 25.3 s (started: 2022-08-31 23:02:21 +00:00)


# Primary processing

In [ ]:
data.shape

(14491324, 3)

time: 5.52 ms (started: 2022-08-31 23:03:14 +00:00)


In [ ]:
# Remove missing values
data = data.dropna()

data = data.rename(columns={"founder_inn": "organisation_inn",
                            'inn': 'participant_id',
                            'capital_p': 'equity_share'})

# columns are renamed for consistency with further code
# data=data[['organisation_inn','participant_id','equity_share']]

# data = data[data['equity_share'] > 0]
# data = data[data.participant_id != data.organisation_inn]


data= data[(data['equity_share'] > 0 )&
          (data.participant_id != data.organisation_inn)]
data.head()

,organisation_inn,participant_id,equity_share
0,041102756227,0408004361,100.0
1,164300048548,1643005423,30.0
2,164301176969,1643005423,10.0
3,164302137200,1643005423,10.0
4,164302594366,1643005423,20.0


time: 5.45 s (started: 2022-08-31 23:03:15 +00:00)


In [ ]:
# normalization of in-edge weights to 1
gdata = data.groupby('organisation_inn').sum().reset_index()
dict_companies = dict(gdata.values)
data['equity_share'] = data['equity_share']/np.array([dict_companies[num] for num in data['organisation_inn']])

ValueError: ignored

time: 32.8 s (started: 2022-08-31 23:07:01 +00:00)


In [ ]:
# finding SH and ST nodes
data['super_holder']=~pd.Series(data.participant_id).isin(data.organisation_inn)
data['super_target']=~pd.Series(data.organisation_inn).isin(data.participant_id)

time: 13.9 s (started: 2022-08-31 23:04:25 +00:00)


In [ ]:
data.head()

,organisation_inn,participant_id,equity_share
0,041102756227,0408004361,0.186916
1,164300048548,1643005423,0.666667
2,164301176969,1643005423,1.000000
3,164302137200,1643005423,1.000000
4,164302594366,1643005423,0.064781


time: 18.5 ms (started: 2022-08-31 22:49:32 +00:00)


# Edges analysis

In [ ]:
e = len(data)
e1 = len(data[(data['super_holder'] == True) & (data['super_target'] == True)])
e2 = len(data[(data['super_holder'] == True) & (data['super_target'] == False)])
e3 = len(data[(data['super_holder'] == False) & (data['super_target'] == True)])
e4 = len(data[(data['super_holder'] == False) & (data['super_target'] == False)])
print('total edges:', e)
print('SH -> ST edges', e1, '({0:.2f}%)'.format(e1/e*100))
print('SH -> ~ST edges', e2, '({0:.2f}%)'.format(e2/e*100))
print('~SH -> ST edges', e3, '({0:.2f}%)'.format(e3/e*100))
print('~SH -> ~ST edges', e4, '({0:.2f}%)'.format(e4/e*100))

total edges: 11709364
SH -> ST edges 10921658 (93.27%)
SH -> ~ST edges 360609 (3.08%)
~SH -> ST edges 376662 (3.22%)
~SH -> ~ST edges 50435 (0.43%)
time: 1.18 s (started: 2022-08-31 22:37:31 +00:00)


# Network analysis

## Core creation

In [ ]:
print('pruning SH and ST nodes of the external layer...')
rdata = data.loc[(data['super_holder'] == False) & (data['super_target'] == False)]
edges_left = len(rdata)
print('layer 1:', edges_left, 'edges left')

crdata = rdata.copy()

for i in range(1, LEVELS):
    is_sh = ~pd.Series(rdata.participant_id).isin(rdata.organisation_inn)
    current_unique_sh = pd.Series(rdata.loc[is_sh == True].participant_id.value_counts().keys())
    print('current SH', len(current_unique_sh))
    curr_sh_prop_name = 'is_level_' + str(i) + '_SH'
    crdata[curr_sh_prop_name] = pd.Series(crdata.participant_id).isin(current_unique_sh)

    is_st = ~pd.Series(rdata.organisation_inn).isin(rdata.participant_id)
    current_unique_st = pd.Series(rdata.loc[is_st == True].organisation_inn.value_counts().keys())
    print('current ST', len(current_unique_st))
    curr_st_prop_name = 'is_level_' + str(i) + '_ST'
    crdata[curr_st_prop_name] = pd.Series(crdata.organisation_inn).isin(current_unique_st)

    rdata = rdata.loc[(is_sh == False) & (is_st == False)]
    print('layer {}: {} edges left'.format(i+1, len(rdata)))

data_core = rdata.copy()
core_holders = rdata.participant_id.value_counts()
print('unique core holders:', len(core_holders))
core_targets = rdata.organisation_inn.value_counts()
print('unique core targets:', len(core_targets))

pruning SH and ST nodes of the external layer...
layer 1: 50435 edges left
current SH 23773
current ST 21961
layer 2: 10539 edges left
current SH 1942
current ST 1560
layer 3: 7117 edges left
current SH 346
current ST 277
layer 4: 6421 edges left
current SH 122
current ST 92
layer 5: 6169 edges left
current SH 64
current ST 27
layer 6: 6038 edges left
current SH 34
current ST 13
layer 7: 5973 edges left
current SH 26
current ST 3
layer 8: 5931 edges left
current SH 24
current ST 4
layer 9: 5889 edges left
current SH 18
current ST 3
layer 10: 5854 edges left
current SH 15
current ST 4
layer 11: 5824 edges left
current SH 11
current ST 3
layer 12: 5808 edges left
current SH 6
current ST 4
layer 13: 5796 edges left
current SH 6
current ST 2
layer 14: 5783 edges left
current SH 5
current ST 3
layer 15: 5773 edges left
current SH 4
current ST 1
layer 16: 5763 edges left
current SH 2
current ST 1
layer 17: 5759 edges left
current SH 2
current ST 0
layer 18: 5756 edges left
current SH 3
curre

# Creating P matrix for core members

Убираем ненужные столбцы, создаем доп столбец `serial_number` для присвоения компании/участнику уникального номера

In [ ]:
data_core_graph = data_core[['organisation_inn', 'equity_share']]
data_core_graph.index = data_core['participant_id']
# we will determine the serial number later
data_core_graph.insert(2, "serial_number", 0)
print(data_core_graph)

               organisation_inn  equity_share  serial_number
participant_id                                              
0245009730           0233004294      1.000000              0
0250008394           0250009239      0.328529              0
0250008404           0250001342      0.123348              0
0250008362           0250009239      0.334200              0
0250008362           0250001342      0.012827              0
...                         ...           ...            ...
7743286458           7721371340      0.204605              0
9731019959           7704465622      0.990000              0
9731019998           7706464857      0.502513              0
9709040271           9718054016      0.500250              0
7751153324           7751001160      1.000000              0

[5751 rows x 3 columns]
time: 21 ms (started: 2022-08-31 22:37:49 +00:00)


## Creating indexes for repeating vertices

Создадим словарь `double_edges` для повторяющихся участников, в котором ключи - индексы в `data_core_graph`, значения - `participant_id` по данному индексу

In [ ]:
def get_key_by_value(dictionary, value):
  for k, v in dictionary.items():
    if v == value:
      return k

double_edges = {}

i = 0
while(len(data_core_graph) > 0 and data_core_graph.index.value_counts()[i] > 1):
  double_edges[i] = (data_core_graph.index.value_counts().index[i])
  i+= 1

print(double_edges)

{0: '7709544821', 1: '7715520956', 2: '7702524304', 3: '7722514294', 4: '7722514262', 5: '7722514431', 6: '7719516340', 7: '7722514400', 8: '7722514350', 9: '7720507848', 10: '7722514382', 11: '7730507205', 12: '5003046281', 13: '9703036046', 14: '7707620724', 15: '0268034331', 16: '7710658157', 17: '7709597566', 18: '7719277981', 19: '7727536647', 20: '7728541343', 21: '7728541336', 22: '7729144130', 23: '7709597453', 24: '7727536460', 25: '7719547148', 26: '7710662717', 27: '0274102068', 28: '7842005676', 29: '1660040227', 30: '7842007151', 31: '7842007698', 32: '0274102050', 33: '7842009744', 34: '6659068890', 35: '7842009769', 36: '7708561302', 37: '1658040540', 38: '1660072042', 39: '7838494350', 40: '2320075200', 41: '4002003036', 42: '5403174513', 43: '1657004451', 44: '7708111423', 45: '6660155957', 46: '7709776420', 47: '0411170866', 48: '5836634295', 49: '5837037686', 50: '7841364175', 51: '1831075222', 52: '7751188020', 53: '1833019311', 54: '0274102082', 55: '4205062622', 5

## Creating serial number in data_core

Проставляем каждому владельцу (`participant_id`) свой идентификационный номер, в случае повторного появления - выставляется `id` первого вхождения

In [ ]:
current_serial_number = 0
index_serial_number = 0

for i in range(len(data_core_graph)):
  if (data_core_graph.index[i] in double_edges.values()):
    # if repeated edge
    current_serial_number = get_key_by_value(double_edges, data_core_graph.index[i])
  else:
    while(index_serial_number in double_edges):
      index_serial_number+= 1
    current_serial_number = index_serial_number
    index_serial_number+= 1

  data_core_graph.loc[data_core_graph.index[i], 'serial_number'] = current_serial_number# try to use apply
data_core_graph.head()

NameError: ignored

# Finding strongly connected components (by Tarjan's Algorithm)

In [ ]:
def get_serial_number_by_vertice(v):
  for i in range(len(data_core_graph)):
    if(data_core_graph.index[i] == v):
      return data_core_graph['serial_number'][i]

time: 1.04 ms (started: 2022-08-31 22:43:39 +00:00)


создаем `table_of_edges`, будем там хранить пары `(i, j)`, означающие, что компания с `serial_number` = `i` владеет компанией с `serial_number` = `j`

In [ ]:
table_of_edges = []
for i in range(len(data_core_graph)):
  table_of_edges.append((get_serial_number_by_vertice(data_core_graph.index[i]), get_serial_number_by_vertice(data_core_graph['organisation_inn'][i])))
print('Total edges: ', len(table_of_edges))

Total edges:  5751
time: 33.3 s (started: 2022-08-31 22:43:43 +00:00)


In [ ]:
from collections import defaultdict
 
def from_edges(edges):
    # translate list of edges to list of nodes
 
    class Node:
        def __init__(self):
            # root is one of:
            # None: not yet visited
            # -1: already processed
            # non-negative integer: lowlink
            self.root = None
            self.successors = []
 
    nodes = defaultdict(Node)
    for v,w in edges:
        nodes[v].successors.append(nodes[w])
 
    for i,v in nodes.items(): # name the nodes for final output
        v.id = i
 
    return nodes.values()
 
def tarjan(V):
    def strongconnect(v, S):
        v.root = pos = len(S)
        S.append(v)
 
        for w in v.successors:
            if w.root is None:  # not yet visited
                yield from strongconnect(w, S)
 
            if w.root >= 0:  # still on stack
                v.root = min(v.root, w.root)
 
        if v.root == pos:  # v is the root, return everything above
            res, S[pos:] = S[pos:], []
            for w in res:
                w.root = -1
            yield [r.id for r in res]
 
    for v in V:
        if v.root is None:
            yield from strongconnect(v, [])

[862, 144, 863, 461, 668]
[864, 1784, 474, 1785]
[221, 232, 233, 1495]
[867, 226, 231, 681]
[868, 15, 869, 870, 871, 872, 873, 874, 875, 876]
[878, 537, 1497, 472]
[880, 362, 1954, 12, 904, 989, 1001, 1016, 1034, 1128, 1129, 1175, 1258, 1563, 4079]
[680, 1496, 2681]
[677, 891, 453, 70, 3174, 3175, 3876, 47]
[643, 3780, 338, 471, 3781]
[647, 900, 649]
[652, 653, 460]
[910, 658, 911]
[60, 2682, 2683, 2684, 2685, 2686]
[912, 1504, 1503]
[914, 2736, 2619]
[660, 915, 529]
[94, 924, 43, 37, 922, 29, 1792, 183, 926, 135, 1505, 1793, 131, 38, 1794]
[919, 421, 3056, 165, 220, 212, 3637, 305]
[687, 719, 723]
[2934, 4045, 588]
[717, 492, 431, 147, 124, 89, 202, 1791, 928, 2362, 203, 418, 2928, 3786, 167, 3788, 3185]
[935, 714, 1508]
[934, 724, 936]
[728, 729, 53, 51, 209, 730, 211, 145]
[943, 656, 1512]
[712, 1513, 1511]
[691, 237, 1987, 186, 218, 1988]
[694, 205, 695, 59, 238, 955]
[55, 558, 559, 380, 447, 381]
[40, 82, 146, 1408, 121, 141, 1539, 143]
[965, 976, 3500, 3792, 2553, 3498]
[969, 152

# Finding the distribution of possessions in one adjacency component

In [ ]:
T_list = []
all_vertices_in_components_of_strong_connectivity = []

def find_stationary(A):
  #note: the matrix is row stochastic.
  #A markov chain transition will correspond to left multiplying by a row vector.
  #We have to transpose so that Markov transitions correspond to right multiplying by a column vector.  np.linalg.eig finds right eigenvectors.
  evals, evecs = np.linalg.eig(A.T)
  evec1 = evecs[:,np.isclose(evals, 1)]

  #Since np.isclose will return an array, we've indexed with an array
  #so we still have our 2nd axis.  Get rid of it, since it's only size 1.
  evec1 = evec1[:,0]
  stationary = evec1 / evec1.sum()

  #eigs finds complex eigenvalues and eigenvectors, so you'll want the real part.
  stationary = stationary.real
  return stationary

def get_vertice_by_serial_number(serial_number):
  return (data_core_graph[data_core_graph['serial_number'] == serial_number].index[0])

def get_A_for_component(component):
  vertices_one_component = {}
  tmp = {'participant_id': [], 'organisation_inn': [], 'equity_share': [], 'serial_number': []}
  cur_data_core_graph = pd.DataFrame(tmp).set_index('participant_id')

  A_cur_matrix = np.zeros((len(component), len(component)), dtype=float)
  
  for i in range(len(component)):
    vertices_one_component[i] = component[i]
    all_vertices_in_components_of_strong_connectivity.append(get_vertice_by_serial_number(component[i]))

  cur_data_core_graph = pd.concat([cur_data_core_graph, data_core_graph[data_core_graph.serial_number.isin(component)]])
  component_inns = set(cur_data_core_graph.index.to_numpy())
  cur_data_core_graph = cur_data_core_graph[cur_data_core_graph.organisation_inn.isin(component_inns)]
  for i in range(len(cur_data_core_graph)):
    a = get_key_by_value(vertices_one_component, get_serial_number_by_vertice(cur_data_core_graph.index[i]))
    b = get_key_by_value(vertices_one_component, get_serial_number_by_vertice(cur_data_core_graph['organisation_inn'][i]))
    A_cur_matrix[a][b] = cur_data_core_graph['equity_share'][i]

  return A_cur_matrix

for component in tarjan(from_edges(table_of_edges)):
  if(len(component) < 2):
        continue
  D = np.zeros((len(component), len(component)), dtype=float)
  A_matrix_one_component = get_A_for_component(component)
  correct_component = True

  for i in range(len(A_matrix_one_component)):
    s = 0
    for j in range(len(A_matrix_one_component)):
      s+= A_matrix_one_component[j][i]
    
    if(not math.isclose(1.0, s)):
      correct_component = False
  
  component_vertices = []
  if(correct_component):
    T_current = np.zeros((len(component), len(component)), dtype=float)
    
    stationary = find_stationary(A_matrix_one_component)

    for m in range(len(component)):
      component_vertices.append(get_vertice_by_serial_number(component[m]))
      for n in range(len(component)):
        T_current[m][n] = stationary[m]
    T_list.append([component_vertices, T_current])
  else:
    B = np.eye(len(A_matrix_one_component)) - A_matrix_one_component
    try:
      T_current = A_matrix_one_component.dot(np.linalg.inv(B))
    except np.linalg.LinAlgError as err:
      print(err)
      print("component = ", component)
      print("A = ", A_matrix_one_component)
      print("B = ", B)
      continue
    for m in range(len(T_current)):
      component_vertices.append(get_vertice_by_serial_number(component[m]))
      cur_col_sum = 0
      
      for n in range(len(T_current)):
        cur_col_sum += T_current[n][m]
      
      if(cur_col_sum == 0):
        print("component = ", component)
      else:
        for n in range(len(T_current)):
          T_current[n][m] = T_current[n][m]/cur_col_sum

    T_list.append([component_vertices, T_current])

# Creating an ancestor hash table for a given vertex

In [ ]:
additional_graph = data.copy(deep=True)
additional_graph.rename(columns = {'super_holder' : 'terminality'}, inplace = True)
additional_graph = additional_graph.drop(columns = ['super_target'], axis = 1)
additional_graph = additional_graph.set_index('organisation_inn')

`ключ` - вершина, `значение` - список пар `(h, s)`, где `h` - владелец (holder), `s` - доля (share).

*Первый элемент списка - вес вершины (конечная доля по отношению к рассматриваемой компании)*

In [ ]:
suitable_vertices = dict()
stack_vertices = []
used_ancestors = set()

def find_Z_by_company_inn(company_inn):
  for i in range(len(T_list)):
    for j in range(len(T_list[i][0])):
      if(T_list[i][0][j] == company_inn):
        return T_list[i], j

def set_suitable_vertices(company_inn: str) -> bool:
    st_time = time.monotonic()
    suitable_vertices.clear()
    stack_vertices.clear()
    used_ancestors.clear()
    if(len(data[data['organisation_inn'] == company_inn]) == 0 and not stack_vertices):
        print("Without owners")
        return False
    
    while(True):
      if(time.monotonic() - st_time > 60.0):
        print("Вычисление хэш-таблицы более 60 секунд...")
        return False
      if(len(data[data.organisation_inn == company_inn]) == 0):
        suitable_vertices[company_inn] = [0] # for terminal vertices, the initial weight is 0
        if(len(stack_vertices) == 0):
            return True
        company_inn = stack_vertices.pop()
        used_ancestors.add(company_inn)
        continue
      
      cur_Z = []
      is_component_ancestors_need_added = True
      is_company_in_strong_component = (company_inn in all_vertices_in_components_of_strong_connectivity)
      
      # if the company is in a strongly connected component,
      # then we will find the ownership component, write it in 'cur_Z'
      # no further actions will happen with 'cur_Z' if the company is not in the component        
      
      if(is_company_in_strong_component):
        cur_Z, index_cur_company_in_Z = find_Z_by_company_inn(company_inn) 

      cur_data = additional_graph[additional_graph.index == company_inn]
      noncomponent_ancestors = [0]
      component_ancestors = [0]
      ancestors = cur_data['participant_id'].values
      
      for ancestor in ancestors:
        if(ancestor not in used_ancestors):
          stack_vertices.append(ancestor)
        cur_ancestor_inn = cur_data[cur_data['participant_id'] == ancestor].participant_id[0]
        if(cur_ancestor_inn == None):
            print("Error related to 'cur_ancestor_inn'")
            return False
        cur_ancestor_share = cur_data[cur_data['participant_id'] == ancestor].equity_share[0]
        if(is_company_in_strong_component):
          if(ancestor in all_vertices_in_components_of_strong_connectivity):
            # cur_company in component and ancestor in component
            if(is_component_ancestors_need_added):
              for j in range(len(cur_Z[0])):
                cur_ancestor_inn = cur_Z[0][j]
                cur_ancestor_share = cur_Z[1][j][index_cur_company_in_Z]
                component_ancestors.append([cur_ancestor_inn+'L', cur_ancestor_share])
              is_component_ancestors_need_added = False
          else:
            # cur_company in component, ancestor not in component
            noncomponent_ancestors.append([cur_ancestor_inn, cur_ancestor_share])
        else:
          if(ancestor in all_vertices_in_components_of_strong_connectivity):
            # cur_company not in component, ancestor in component
            noncomponent_ancestors.append([cur_ancestor_inn+'R', cur_ancestor_share])
          else:
            # cur_company not in component, ancestor not in component
            noncomponent_ancestors.append([cur_ancestor_inn, cur_ancestor_share])
      
      if(is_company_in_strong_component):
        suitable_vertices[company_inn+'R'] = component_ancestors
        suitable_vertices[company_inn+'L'] = noncomponent_ancestors
      else:
        suitable_vertices[company_inn] = noncomponent_ancestors

      if(stack_vertices):
        company_inn = stack_vertices.pop()
        used_ancestors.add(company_inn)
      else:
        return True

# REQUEST

In [ ]:
requested_company = "0714001578"
set_suitable_vertices(requested_company)

True

для `левых вершин в двудольном графе`, у которых сумма входящих владений `< 1`, но `> 0` (то есть кто-то владеет, но не на 100%) -- строим дополнительную вершину без `L` или `R`

In [ ]:
def set_additional_vertex():
  global suitable_vertices
  suitable_vertices_copy = suitable_vertices.copy()
  for k, v in suitable_vertices.items():
    if(k[len(k)-1] == 'L'):
      s = 0
      for i in range(1, len(v)):
        s += v[i][1]
      if((not math.isclose(s, 1.0)) & (not math.isclose(s, 0))):
        suitable_vertices_copy[k[:len(k)-1]] = [1]
        tmp = v.copy()
        tmp.append([k[:len(k)-1], round(1-round(s,7), 7)])
        suitable_vertices_copy[k] = tmp
  suitable_vertices = suitable_vertices_copy
  
set_additional_vertex()

добавляем последним элементом списка `T`- вершина терминальна, `N` - вершина нетерминальна

In [ ]:
def get_vertex_without_RL(company_inn: str)->str:
  if(company_inn[len(company_inn)-1] == 'R' or company_inn[len(company_inn)-1] == 'L'):
    return str(company_inn[:len(company_inn)-1])
  else:
    return company_inn

def set_terminality_to_table(company_inn: str):
  global suitable_vertices
  suitable_vertices_copy = suitable_vertices.copy()
  for k, v in suitable_vertices.items():
    s = 0
    for i in range(1, len(v)):
      s += v[i][1]
    if(s == 0.0 and (k[len(k)-1] != 'R')):
      v.append('T')
      v[0] = 0
    else:
      v.append('N')
      if(get_vertex_without_RL(k) == company_inn and (k[len(k)-1] != 'L')):
        v[0] = 1
      else:
        v[0] = 0
    # print(k, " ", v)
  suitable_vertices = suitable_vertices_copy
    
set_terminality_to_table(requested_company)

# Algorithm for finding the correct weights for each ancestor vertex

In [ ]:
from collections import deque

queue_vertices = deque()
final_owners = dict()

def get_vertex_name_by_presence(company_inn: str) -> str:
  if(suitable_vertices.get(company_inn+'R') == None):
    # if not in the component
    if(suitable_vertices.get(company_inn) != None):
      # if not in the component, but there is such a vertex
      return company_inn
    else:
      return ""
  else:
    # if in the component, then we will start the crawl from the RIGHT lobe
      return company_inn+'R'

def get_equity_share(company_inn: str) -> bool:
  st_time = time.monotonic()
  queue_vertices.clear()
  final_owners.clear()
  queue_vertices.append(get_vertex_name_by_presence(company_inn))
  s = 0
  while(queue_vertices):
    if(time.monotonic() - st_time > 60.0):
      print("Вычисление конечных владельцев более 60 секунд...")
      return False
    cur_company = queue_vertices.pop()
    company_info_dict = suitable_vertices.get(cur_company)

    if(company_info_dict == None):
      print("Couldn't find the entered company")
      return False

    # company_info_dict[0] = vertex weight (ownership share)
    # company_info_dict[1...len(company_info_dict)-2] = (owner, equity_share)
    # company_info_dict[len(company_info_dict)-1] = terminality
    for i in range(1, len(company_info_dict)-1):
      cur_owner = company_info_dict[i][0]
      cur_equity_share = company_info_dict[i][1]
      owner_info_dict = suitable_vertices.get(cur_owner)
      if(owner_info_dict == None):
        print(f"An error occurred with the owner search {cur_owner}")
        return False

      owner_info_dict[0] = company_info_dict[0] * cur_equity_share
      if(owner_info_dict[len(owner_info_dict)-1] == "T"):
        s+= owner_info_dict[0]
        cur_owner = get_vertex_without_RL(cur_owner)
        if(cur_owner in final_owners):
          # if we considered the path to this vertex, then we sum it up with the current weight
          final_owners.update({cur_owner: owner_info_dict[0] + final_owners.get(cur_owner)})
        else:
          # if you met for the first time, then we set the ownership share
          final_owners[cur_owner] = owner_info_dict[0]
      else:
        queue_vertices.append(cur_owner)
  # presentation from the most important owners to the smaller ones
  list_owners = list(final_owners.items())
  list_owners.sort(key=lambda i: i[1])
  list_owners.reverse()
  owner_counter = 1
  print(f"Ownership share in the company {company_inn}:")
  for owner in list_owners:
    print(f'{owner_counter}. {owner[0]} = {(owner[1]*100*(1.0/s)):.4f}%')
    owner_counter+= 1
  print(f"The total amount of ownership share is equal to {(s*100*(1.0/s)):.9}%")
  return True

get_equity_share(requested_company)

Ownership share in the company 0714001578:
1. 0703008500 = 22.4747%
2. 0703008395 = 22.4511%
3. 0703008469 = 18.6050%
4. 0703008490 = 11.4500%
5. 0703008300 = 6.5610%
6. 0703008772 = 6.2433%
7. 0714001578 = 3.5928%
8. 0703008370 = 3.3776%
9. 0726000378 = 2.8737%
10. 0726019001 = 2.3708%
The total amount of ownership share is equal to 100.0%


True